# Chat with chat history

- metaprompts
https://docs.google.com/presentation/d/1fsAaNgBo32JjAHl3s5tqyQ7fo5niqO3NYDtvkA3ZbGg/edit#slide=id.g2b7297b503f_0_63


### Initialize the kernel

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.7.1"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.7.1-alpha"

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

#!import Settings.cs

using Microsoft.SemanticKernel;

var kernel = Settings.InitializeKernel();

Installed Packages Microsoft.SemanticKernel, 1.7.1 Microsoft.SemanticKernel.Plugins.Core, 1.7.1-alpha

### Initialize Chat Completion service

In [2]:

var chatGPT = kernel.GetRequiredService<IChatCompletionService>();


### Initialize chat history

In [3]:
using Microsoft.SemanticKernel.ChatCompletion;

var systemMessage = "You are a librarian, expert about books";
var chatHistory = new ChatHistory(systemMessage);

// or in a more compact form
//var chatHistory = new ChatHistory("You are a librarian, expert about books");


### Let's rebuild a chat nistory by adding some user and assistants messages to it. Please notice that we have two types of messages below: user and assistant messsages.
### Do not forget that these are not requesting a response from the model below

In [4]:
chatHistory.AddUserMessage("Hi, I'm looking for book suggestions");

### And now an assistant message (in fact, a response from model)

In [5]:

chatHistory.AddAssistantMessage("Great, I'd be happy to help with book suggestions. To better assist you, could you let me know what genres or types of books you're interested in? Some further details such as your favorite books or authors, themes you enjoy, or what mood you're in for reading could help me provide more tailored recommendations.");

### One more message

In [6]:

chatHistory.AddUserMessage("I love history and philosophy, I'd like to learn something new about Greece, any suggestion?");

### Let's inspect the chat history so far

In [7]:

foreach (var message in chatHistory)
{
    Console.WriteLine($"{message.Role} > {message.Content}");
}

system > You are a librarian, expert about books
user > Hi, I'm looking for book suggestions
assistant > Great, I'd be happy to help with book suggestions. To better assist you, could you let me know what genres or types of books you're interested in? Some further details such as your favorite books or authors, themes you enjoy, or what mood you're in for reading could help me provide more tailored recommendations.
user > I love history and philosophy, I'd like to learn something new about Greece, any suggestion?


### Now we ask the model for a response (we can notice the reply from the assistant message)

In [8]:
var reply = await chatGPT.GetChatMessageContentAsync(chatHistory);
chatHistory.Add(reply);

$"{chatHistory[^1].Role} > {chatHistory[^1].Content}"

assistant > Certainly! If you're interested in the history and philosophy of Ancient Greece, there are many fascinating books that could broaden your knowledge and provoke thought. Here's a mix of classic texts and modern histories for you to consider:

1. **"The Histories" by Herodotus** - Often considered the "Father of History," Herodotus provides an account of the Greco-Persian Wars and offers insights into the cultures and events of the ancient world.

2. **"The Peloponnesian War" by Thucydides** - This classic text gives a detailed historical account of the war between Sparta and Athens in the 5th century BCE and is also a deep investigation into the nature of power politics.

3. **"The Republic" by Plato** - Not only is this a central text for understanding Platonic philosophy, but it also provides a lens into Greek views on justice, governance, and society.

4. **"Nicomachean Ethics" by Aristotle** - In this philosophical work, Aristotle outlines his thoughts on virtue, charact

### Oops, where did 'display' command dissapear?

### We can ask the model for a response in streaming flavour

In [9]:
Console.Write($"{Azure.AI.OpenAI.ChatRole.Assistant} > ");
await foreach (var content in chatGPT.GetStreamingChatMessageContentsAsync(chatHistory))
{
    Console.Write(content);
}

assistant > Certainly! If you're interested in the history and philosophy of Ancient Greece, there are many fascinating books that could broaden your knowledge and provoke thought. Here's a mix of classic texts and modern histories for you to consider:

1. **"The Histories" by Herodotus** - Often considered the "Father of History," Herodotus provides an account of the Greco-Persian Wars and offers insights into the cultures and events of the ancient world.

2. **"The Peloponnesian War" by Thucydides** - This classic text gives a detailed historical account of the war between Sparta and Athens in the 5th century BCE and is also a deep investigation into the nature of power politics.

3. **"The Republic" by Plato** - Not only is this a central text for understanding Platonic philosophy, but it also provides a lens into Greek views on justice, governance, and society.

4. **"Nicomachean Ethics" by Aristotle** - In this philosophical work, Aristotle outlines his thoughts on virtue, charact